In [1]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
import json


In [2]:
import io
# import pandas as pd
# from google.colab import files
from collections import OrderedDict
import inspect as inspector
import sys
import argparse
# import json
import time
import random
import hashlib

In [3]:
!pip install -q "tensorflow-text"

import requests
import tensorflow as tf
import tensorflow_text as tf_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 52.3 MB/s eta 0:00:00


In [4]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses

!pip install transformers #==2.9.0
!pip install torch  #==2.0.0

# !pip install https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.1.0-py3-none-any.whl

!pip install accelerate
# !pip install transformers==2.9.0 
!pip install pytorch_lightning  #==0.7.5
# !pip install datsets transformers[sentencepiece]
!pip install sentencepiece

!pip install Sentencepiece
# !pip install transformers

!pip install accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 65.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.0 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=b4b8ed9b3d913ed86a672d3fa3536cd611b14b8fbd55ec52b0e6d9ef35c711fd
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 94.5

In [5]:
data_path = './train.jsonl'
df = pd.read_json(data_path, lines=True)
df_validation = pd.read_json('./validation.jsonl', lines=True)

In [6]:
def concatenate_strings(lst):
    separator = ' '
    return separator.join(lst)

In [7]:
df['targetParagraphs'] = df['targetParagraphs'].apply(concatenate_strings)
df['postText'] = df['postText'].apply(concatenate_strings)
df['spoiler'] = df['spoiler'].apply(concatenate_strings)
df['tags'] = df['tags'].apply(concatenate_strings)

In [8]:
df_validation['targetParagraphs'] = df_validation['targetParagraphs'].apply(concatenate_strings)
df_validation['postText'] = df_validation['postText'].apply(concatenate_strings)
df_validation['spoiler'] = df_validation['spoiler'].apply(concatenate_strings)
df_validation['tags'] = df_validation['tags'].apply(concatenate_strings)

In [9]:
def generate_new_df(df):

    new_df = pd.DataFrame({'tags': df['tags'],'postText':df['postText'],'paragraph' : df['targetParagraphs'] ,'spoiler': df['spoiler']})
    return new_df
    

In [10]:

# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt



# sklearn

from sklearn.metrics import confusion_matrix, classification_report

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer



# Utility

import re
import string
import pickle

In [11]:
wordLemm = WordNetLemmatizer()
def processed_postText(paragraph):
    # Lower Casing
    paragraph = paragraph.lower()
    # Remove punctuations
    paragraph = paragraph.translate(str.maketrans("","",string.punctuation))
    # Tokenizing words
    tokens = word_tokenize(paragraph)
    # Removing Stop Words
    #final_tokens = [w for w in tokens if w not in stopword]
    # Reducing a word to its word stem using lemmatization
    
    return ' '.join(tokens)

In [12]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [13]:
nltk.download('stopwords')
stopword = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
new_df = generate_new_df(df)

In [15]:
new_df_validation = generate_new_df(df_validation)

In [16]:
 new_df['paragraph'] = new_df['paragraph'].apply(lambda x: processed_postText(x))
 new_df['postText'] = new_df['postText'].apply(lambda x: processed_postText(x))
# print('Text Preprocessing complete.')

In [17]:
 new_df_validation['paragraph'] = new_df_validation['paragraph'].apply(lambda x: processed_postText(x))
 new_df_validation['postText'] = new_df_validation['postText'].apply(lambda x: processed_postText(x))

In [18]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
questions= new_df['postText']
answers= new_df['spoiler']
stories_not_summarized = new_df['paragraph']
tags = new_df['tags']

print(stories_not_summarized[0])




it ’ ll be just like old times this weekend for tom brady and wes welker welker revealed friday morning on a miami radio station that he contacted brady because he ’ ll be in town for sunday ’ s game between the new england patriots and miami dolphins at gillette stadium it seemed like a perfect opportunity for the two to catch up but brady ’ s definition of catching up involves far more than just a meal in fact it involves some literal catching as the patriots quarterback looks to stay sharp during his fourgame deflategate suspension i hit him up to do dinner saturday night he ’ s like ‘ i ’ m going to be flying in from ann arbor later after the michigancolorado football game but how about that morning we go throw ’ welker said on wqam per the boston globe and i ’ m just sitting there i ’ m like ‘ i was just thinking about dinner but yeah sure i ’ ll get over there early and we can throw a little bit ’ welker was one of brady ’ s favorite targets for six seasons from 2007 to 2012 it ’

In [20]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 kB 25.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=fe63196ea4232c45dc135e2b6936743da4402b7d7f19853950c3e9b1ca3393e7
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [21]:
import torch

In [ ]:
# from transformers import pipeline
# import requests
# import pprint
# import time
# pp = pprint.PrettyPrinter(indent=14)

# from transformers import T5Tokenizer, T5ForConditionalGeneration
# import wandb

# tokenizer = T5Tokenizer.from_pretrained("t5-small")
# model = T5ForConditionalGeneration.from_pretrained('t5-small')


# # model = T5ForConditionalGeneration.from_pretrained('t5-small')
# # tokenizer = T5Tokenizer.from_pretrained('t5-small')
# device_cuda = torch.device('cuda')

In [22]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [23]:
from datetime import datetime

In [24]:
if WANDB_INTEGRATION:
    wandb_run = wandb.init(
        project="bart_wiki_lingua",
        config={
            "per_device_train_batch_size": 4
            # "learning_rate": training_args.learning_rate
            # "dataset": "wiki_lingua " + "english",
        },
    )

    now = datetime.now()
    current_time = now.strftime("%H%M%S")
    wandb_run.name = "run_" + "english" + "_" + current_time

wandb: Currently logged in as: pradhaneva94 (evapradhan). Use `wandb login --relogin` to force relogin


In [ ]:
# # summarize with T5
# # summarizer_t5 = pipeline(task='summarization', model="t5-large")
# summarizer_t5 = pipeline(task='summarization', model="t5-small")

In [ ]:
#FINE TUNING START

In [25]:
!pip install transformers #==2.9.0
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.3 MB/s eta 0:00:00


In [26]:
from datasets import Dataset
from datasets import dataset_dict

In [27]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
import torch
import numpy as np
import pandas as pd
from transformers import (TrainingArguments, Trainer, EarlyStoppingCallback) #AutoTokenizer, AutoModelForSequenceClassification,
from transformers import BartTokenizer, BartForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score
from datasets import Dataset, load_metric
# from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
from transformers import AutoTokenizer, LongT5ForConditionalGeneration

In [76]:
# max_input = 512
# max_target = 128
# batch_size = 3
model_checkpoints = "t5-small"

In [77]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)

In [44]:
input_text = "Summarize:" + " tags:" + new_df["tags"] + " condition: " + new_df['postText'] + " paragraph: " + new_df["paragraph"]
target_text = new_df["spoiler"]


data_to_process_train = pd.DataFrame({ 'input_ids': input_text, 'labels':target_text})

In [45]:
data_to_process_train

,input_ids,labels
0,Summarize: tags:passage condition: wes welker ...,how about that morning we go throw?
1,Summarize: tags:phrase condition: nasa sets da...,2070
2,Summarize: tags:phrase condition: this is what...,intellectual stimulation
3,Summarize: tags:multi condition: passion is ov...,Purpose connects us to something bigger and in...
4,Summarize: tags:phrase condition: the perfect ...,in a rice cooker
...,...,...
3195,Summarize: tags:passage condition: has faceboo...,it hasn’t necessarily taken the wind out of Yo...
3196,Summarize: tags:passage condition: cop is eati...,"It read, ""Thanks for keeping us safe."""
3197,Summarize: tags:multi condition: 5 popular myt...,1. Anti-wrinkle creams will erase the fine lin...
3198,Summarize: tags:phrase condition: you need to ...,@beyoncefan666


In [46]:
df1_dict = data_to_process_train.to_dict(orient = 'list')

In [47]:
dict_train_ = Dataset.from_dict(df1_dict)

In [48]:
dict_train_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 3200
})

In [59]:
input_text = "Summarize:" + " tags:" + new_df_validation["tags"] + " condition: " + new_df_validation['postText'] + " paragraph: " + new_df_validation["paragraph"]
target_text = new_df_validation["spoiler"]

data_to_process_validation = pd.DataFrame({ 'input_ids': input_text, 'labels':target_text})

In [60]:
data_to_process_train

,input_ids,labels
0,Summarize: tags:passage condition: wes welker ...,how about that morning we go throw?
1,Summarize: tags:phrase condition: nasa sets da...,2070
2,Summarize: tags:phrase condition: this is what...,intellectual stimulation
3,Summarize: tags:multi condition: passion is ov...,Purpose connects us to something bigger and in...
4,Summarize: tags:phrase condition: the perfect ...,in a rice cooker
...,...,...
3195,Summarize: tags:passage condition: has faceboo...,it hasn’t necessarily taken the wind out of Yo...
3196,Summarize: tags:passage condition: cop is eati...,"It read, ""Thanks for keeping us safe."""
3197,Summarize: tags:multi condition: 5 popular myt...,1. Anti-wrinkle creams will erase the fine lin...
3198,Summarize: tags:phrase condition: you need to ...,@beyoncefan666


In [61]:
df2_dict = data_to_process_validation.to_dict(orient = 'list')

In [63]:
dict_validation_ = Dataset.from_dict(df2_dict)

In [68]:
dict_validation_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 800
})

In [64]:
input_text = "Summarize:" + " tags:" + new_df_validation["tags"] + " condition: " + new_df_validation['postText'] + " paragraph: " + new_df_validation["paragraph"]
target_text = new_df_validation["spoiler"]

data_to_process_test = pd.DataFrame({ 'input_ids': input_text, 'labels':target_text})

In [65]:
data_to_process_test

,input_ids,labels
0,Summarize: tags:passage condition: five nights...,some of the plot elements are so disturbing th...
1,Summarize: tags:multi condition: why arizona s...,"""intentionally"" could transform a court case a..."
2,Summarize: tags:phrase condition: here ’ s how...,20%
3,Summarize: tags:multi condition: harry potter ...,Alan Rickman & Rupert Grint CBGB
4,Summarize: tags:passage condition: a man swall...,a man who swallowed a 64GB microSD card and th...
...,...,...
795,Summarize: tags:passage condition: this is wha...,The video below shows the stunned cleaner init...
796,Summarize: tags:phrase condition: this texas g...,Christopher Suprun
797,Summarize: tags:multi condition: this beauty q...,Rachel Crawley High fat vegan plant based diet...
798,Summarize: tags:passage condition: wikileaks j...,Julian Assange’s internet link has been intent...


In [66]:
df3_dict = data_to_process_test.to_dict(orient = 'list')

In [67]:
dict_test_ = Dataset.from_dict(df3_dict)

In [69]:
dict_test_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 800
})

In [ ]:
# df1_dict = data_to_process_train.to_dict(orient = 'list')
# df2_dict = data_to_process_validation.to_dict(orient = 'list')
# df3_dict = data_to_process_test.to_dict(orient = 'list')


# dict_train_ = Dataset.from_dict(df1_dict)
# dict_validation_ = Dataset.from_dict(df2_dict)
# dict_test_ = Dataset.from_dict(df3_dict)

In [70]:
dict_all_2 = dataset_dict.DatasetDict({"train":dict_train_, "validation":dict_validation_, "test": dict_test_})

In [71]:
dict_all_2

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 3200
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 800
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 800
    })
})

In [72]:
dict_train_

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 3200
})

In [74]:
def preprocess_data(example):
    para = example['input_ids']
    spoiler = example['labels']

    input_ids = tokenizer(para, padding="max_length", truncation=True, max_length= 1000).input_ids

    with tokenizer.as_target_tokenizer():
      target_ids = tokenizer(spoiler, padding='max_length', truncation=True, max_length= 500).input_ids
    
    return {"input_ids": input_ids, "labels": target_ids}


In [78]:
train_dataset = dict_train_
test_dataset = dict_validation_

train_dataset_processed = train_dataset.map(preprocess_data)
test_dataset_processed = test_dataset.map(preprocess_data)

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [79]:
train_dataset_processed

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 3200
})

In [80]:
test_dataset_processed

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 800
})

In [81]:
from transformers import pipeline
import requests
import pprint
import time
pp = pprint.PrettyPrinter(indent=14)

from transformers import T5Tokenizer, T5ForConditionalGeneration
# import wandb

# tokenizer = T5Tokenizer.from_pretrained("t5-small")
# model = T5ForConditionalGeneration.from_pretrained('t5-small')

In [82]:
model_name = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name, num_labels = 4)

In [83]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.6 MB/s eta 0:00:00


In [84]:
bleu_metric = load_metric("bleu")
meteor_metric = load_metric("meteor")
bertscore_metric = load_metric("bertscore")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [85]:
#out_1 = []
#out_2 = []

def compute_metrics(eval_pred):
    logits_full, labels = eval_pred

    #out_1.append(logits_full[0])
    #out_2.append(logits_full[1])

    predicted_token_ids = torch.argmax(torch.tensor(logits_full[0]), dim=-1).tolist()
    decoded_preds = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds_bleu = [pred.split() for pred in decoded_preds]
    decoded_labels_bleu = [[pred.split()] for pred in decoded_labels]
    
    bleu_score = bleu_metric.compute(predictions=decoded_preds_bleu, references= decoded_labels_bleu)
    meteor_score = meteor_metric.compute(predictions=decoded_preds, references=decoded_labels)
    bertscore_score = bertscore_metric.compute(predictions=decoded_preds, references=decoded_labels, lang = 'en')
    
    return {"bleu": bleu_score["bleu"], "meteor": meteor_score["meteor"], "bertscore": np.mean(bertscore_score["f1"])}

In [86]:
import shutil

directory_path = f"T5_finetuning_results_conditional_all_sizelimited"

# shutil.rmtree(directory_path)

In [87]:
training_args = TrainingArguments(
    output_dir=f"/content/drive/MyDrive/T5_finetuning_results_conditional_all_sizelimited",
    num_train_epochs=25,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    # gradient_accumulation_steps=2,
    evaluation_strategy="epoch",
    logging_steps=100,
    # save_steps=0,
    # load_best_model_at_end=True,
    # metric_for_best_model="bleu",
    # greater_is_better=True,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=3,
    # predict_with_generate=True,
    fp16=True,
    # save_strategy="epoch"

)

# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience= 3)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_processed,
    eval_dataset=test_dataset_processed,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    # compute_metrics=compute_metrics
)


In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [ ]:
metrics = trainer.evaluate()


In [ ]:
metrics

{'eval_loss': 0.0583157055079937,
 'eval_runtime': 5.2539,
 'eval_samples_per_second': 152.267,
 'eval_steps_per_second': 38.067,
 'epoch': 16.0}

In [ ]:
dict_validation_["labels"]

In [ ]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["labels"],
        padding="max_length",
        truncation=True,
        max_length=1000,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

In [ ]:
# model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)
summaries_after_tuning_all = []

def generate_batch(lst, batch_size):
    """  Yields batch of specified size """
    for i in range(0, len(lst), batch_size):
        yield lst[i : i + batch_size]
        # summaries_after_tuning = generate_summary(test_samples, model)[1]
        # summaries_after_tuning_all.append(summaries_after_tuning)


# test_samples = dict_validation_.select(range())
test_samples = generate_batch(dict_validation_, 16)

# # summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]

for batch in test_samples:
  # print(batch)
  summaries_after_tuning = generate_summary(batch, model)[1]
  summaries_after_tuning_all.append(summaries_after_tuning)

In [ ]:
summaries_after_tuning_all

[['Einige der plot elements are so disturbing that they make him feel sick.',
  '"intentionally" could transform a court case against Phoenix-area Sheriff Joe Arpai',
  '20%',
  'Alan Rickman & Rupert Grint CBGB',
  'a man who swallowed a 64GB microSD card and then pooped',
  'Sprite',
  'Smoky Paprika-Baked Garbanzo Beans',
  'McGonagall, a former assistant of Dumbledore, was appointed as',
  'Alle Szenen sind eigentlich in dem Film.',
  '"I had fake relationships, fake fights, I don\'t care anymore, I can',
  'Elettra Wiedemann extra strength work, so weights, and quite a few',
  'Anthony Bourdain',
  "he'd eaten a peanut butter sandwich and wasn't aware of her peanut allergy",
  'Marcia Clark Does she think Simpson really did it?',
  'Mitzi Gaynor Beverly Hills, California',
  'Dibrom'],
 ['They don’t fart fart.',
  "'Kidda', 'Kidda', ",
  'Dunham picked the boy up and took him to a Subway to get something to ',
  'Aubrey a non seulement cérébral palsy, mais elle',
  'beginnt später

'Einige der plot elements are so disturbing that they make him feel sick.'

In [ ]:
stories_list =[]
for i in range(len(summaries_after_tuning_all)):
  for j in range(len(summaries_after_tuning_all[i])):
    summary=summaries_after_tuning_all[i][j]
    stories_list.append(summary)


print(stories_list)

In [ ]:
# type(new_df['spoiler'][0:40])

pandas.core.series.Series

In [ ]:
type(stories_list[0:40])

list

In [ ]:
stories_list_series = pd.Series((v for v in stories_list))

In [ ]:
import pandas as pd

In [ ]:
dataframe = pd.DataFrame({ 'postText': new_df_validation["postText"], 'actual_spoiler':new_df_validation['spoiler'], 'gen_spoiler':stories_list_series })

In [ ]:
dataframe

,postText,actual_spoiler,gen_spoiler
0,five nights at freddy ’ s sequel delayed for w...,some of the plot elements are so disturbing th...,Einige der plot elements are so disturbing tha...
1,why arizona sheriff joe arpaio ’ s fate could ...,"""intentionally"" could transform a court case a...","""intentionally"" could transform a court case a..."
2,here ’ s how much you should be tipping your h...,20%,20%
3,harry potter alums reunite for new movie,Alan Rickman & Rupert Grint CBGB,Alan Rickman & Rupert Grint CBGB
4,a man swallowed a microsd card and you wont be...,a man who swallowed a 64GB microSD card and th...,a man who swallowed a 64GB microSD card and th...
...,...,...,...
795,this is what happens when you leave a hotel cl...,The video below shows the stunned cleaner init...,The video below shows the stunned cleaner init...
796,this texas gop elector announces that he wont ...,Christopher Suprun,Christopher Suprun
797,this beauty queen cured her acne with one diet...,Rachel Crawley High fat vegan plant based diet...,Rachel Crawley High fat vegan plant-based diet...
798,wikileaks julian assange reported dead,Julian Assange’s internet link has been intent...,Julian Assange’s Internetlink has been intenti...


In [ ]:
if WANDB_INTEGRATION:
    wandb_run.finish()

eval/loss,█▅▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▃▆▃▃▃▆▃▂▆▃▃▃▅▅▅█▅▆▅▄▄▄▆▅▄▄▃▃▃▅▁▃▂▃▄▃▁▅▃▄
eval/samples_per_second,▆▃▆▆▆▃▆▇▃▆▆▅▄▄▄▁▄▃▄▅▅▅▂▄▅▅▆▆▆▄▇▆▇▆▅▆█▄▅▅
eval/steps_per_second,▆▃▆▆▆▃▆▇▃▆▆▅▄▄▄▁▄▃▄▅▅▅▂▄▅▅▆▆▆▄▇▆▇▆▅▆█▄▅▅
train/epoch,▁▂▂▃▃▃▄▄▅▁▂▂▃▃▄▄▄▅▅▆▆▇▇█▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train/global_step,▁▂▂▃▃▃▄▄▅▁▂▂▃▃▄▄▄▅▅▆▆▇▇█▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train/learning_rate,█▇▆▅▅▄▃▂▁██▇▇▆▆▅▅▄▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▄▃▃▃▂▃▂▂▃▂▂▂▂▂▂▂▂▁▂▂▁▁▂▂▂▂▂▁▁▁▂▂▂▂▁▃▂▂▁
train/total_flos,▁██
train/train_loss,█▁▁
train/train_runtime,▁██


In [ ]:
#EVALUATION METRICS

In [ ]:
import nltk
import nltk.translate.bleu_score as bleu

import math
import numpy
import os

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')

In [ ]:
bleu_scores = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  score_ref = bleu.sentence_bleu([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"])
  bleu_scores.append(score_ref)



In [ ]:
bleu_scores

[0.7672486789360017,
 0.6391801085391197,
 1.2213386697554703e-77,
 1.0,
 0.7378576076067946,
 1.0,
 1.0,
 0.553871069098511,
 0.14372803312372198,
 0.8340025666944847,
 0.37825041055641406,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.6703200460356393,
 1.1434362398096986e-231,
 0.5501496903546678,
 0.2239981704622798,
 3.1923134147786836e-78,
 0.432385208462013,
 0.9793821813312402,
 0.4656491682341082,
 0.8172460049207133,
 1.0,
 0.22298967204975628,
 0.9881658194110153,
 0.30432118818065745,
 0.4116245235860851,
 0.8488605289566383,
 1.0,
 1.0,
 0,
 0.24543831283640097,
 0.4478519124357522,
 0.6425503166524515,
 0.11627269232057053,
 0.4142332818795304,
 1.0,
 0.17487516229456845,
 0.14920276910823296,
 0.5555238068023582,
 0,
 1.4488496539373276e-231,
 0.7713920621476292,
 0.3583246752499035,
 0.8337411129887954,
 0.7828785637123031,
 0.9775290462529921,
 0.8153551038173115,
 0.216713321183593,
 0.8633400213704505,
 1.0,
 1.0,
 0.4760241069928002,
 1.0,
 1.0,
 0.9337597913454168,
 1.0,
 1.0,


In [ ]:
bleu_average = round(sum(bleu_scores) / len(bleu_scores), 2)
bleu_average

0.58

In [ ]:
bleu_min = min(bleu_scores)
bleu_min

0

In [ ]:
bleu_max = max(bleu_scores)
bleu_max

1.0

In [ ]:
bleu_scores.sort()
mid = len(bleu_scores) // 2
res = (bleu_scores[mid] + bleu_scores[~mid]) / 2
bleu_median = res
bleu_median

0.5960124922620527

In [ ]:
import sys
from nltk.translate.meteor_score import meteor_score

In [ ]:
meteor_scores = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"].strip()

  ref = word_tokenize(gen_spoiler_strip)
  hypo = word_tokenize(dataframe.iloc[i]["actual_spoiler"].strip())
  meteor = nltk.translate.meteor_score.meteor_score([ref], hypo)


  # meteor = nltk.translate.meteor_score.meteor_score([gen_spoiler_strip], dataframe.iloc[i]["actual_spoiler"].strip())
  # meteor = round(meteor_score([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"]), 2) # list of references
  meteor_scores.append(meteor)



In [ ]:
meteor_scores

[0.7613856163179182,
 0.8690693430656935,
 0.9375,
 0.9976851851851852,
 0.9417137886037409,
 0.5,
 0.9921875,
 0.46263586956521746,
 0.125,
 0.9176300578034681,
 0.820141397064474,
 0.9375,
 0.9998518518518519,
 0.9995,
 0.9976851851851852,
 0.5,
 0.711764705882353,
 0.0,
 0.8259259259259261,
 0.31562500000000004,
 0.0,
 0.8972723704866562,
 0.9169724770642201,
 0.7823485876403108,
 0.625,
 0.9375,
 0.44834307992202727,
 0.9299633936261843,
 0.6763688843164152,
 0.8522727272727273,
 0.9171467764060356,
 0.9375,
 0.9814814814814815,
 0.0,
 0.6545159386068478,
 0.9109756097560976,
 0.9814814814814815,
 0.4652205163822554,
 0.8108006890869597,
 0.9921875,
 0.5686936936936937,
 0.05050505050505051,
 0.25,
 0.0,
 0.3333333333333333,
 0.7545454545454545,
 0.5250974658869396,
 0.6388888888888888,
 0.8928571428571429,
 0.9240225015626085,
 0.25,
 0.6222271517302574,
 0.5,
 0.9814814814814815,
 0.9814814814814815,
 0.8913611740992695,
 0.9375,
 0.9375,
 0.9921405664212475,
 0.9921875,
 0.9375,

In [ ]:
meteor_scores_average = round(sum(meteor_scores) / len(meteor_scores), 2)
meteor_scores_average

0.65

In [ ]:
meteor_scores_max = max(meteor_scores)
meteor_scores_max

0.9999375

In [ ]:
meteor_scores_min = min(meteor_scores)
meteor_scores_min

0.0

In [ ]:
meteor_scores.sort()
mid = len(meteor_scores) // 2
res = (meteor_scores[mid] + meteor_scores[~mid]) / 2
meteor_scores_median = res
meteor_scores_median

0.7809475806451613

In [ ]:
!pip install bert-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.0 MB/s eta 0:00:00


In [ ]:
from bert_score import score

In [ ]:
!git clone https://github.com/Tiiiger/bert_score.git

Cloning into 'bert_score'...
remote: Enumerating objects: 993, done.
remote: Counting objects: 100% (355/355), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 993 (delta 219), reused 322 (delta 198), pack-reused 638
Receiving objects: 100% (993/993), 1.34 MiB | 10.33 MiB/s, done.
Resolving deltas: 100% (534/534), done.


In [ ]:
bert_scores_P = []
bert_scores_R = []
bert_scores_F1 = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"]
  actual_spoiler_strip = dataframe.iloc[i]["actual_spoiler"]


  ref = word_tokenize(gen_spoiler_strip)
  hypo = word_tokenize(actual_spoiler_strip)

  P, R, F1 = score(gen_spoiler_strip, actual_spoiler_strip, lang="en", verbose=True)
  bert_scores_P.append(P)
  bert_scores_R.append(R)
  bert_scores_F1.append(F1)

  # meteor = nltk.translate.meteor_score.meteor_score([gen_spoiler_strip], dataframe.iloc[i]["actual_spoiler"].strip())
  # meteor = round(meteor_score([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"]), 2) # list of references
  # meteor_scores.append(meteor)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 4>:13                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/bert_score/score.py:70 in score                          │
│                                                                                                  │
│    67 │   │   │   │     multiple references, the returned score of this candidate is             │
│    68 │   │   │   │     the *best* score among all references.                                   │
│    69 │   """                                                                                    │
│ ❱  70 │   assert len(cands) == len(refs), "Different number of candidates and references"        │
│    71 │                                                                                          │
│    72 │   assert (                                                                               │
│    73 │   │   lang is not None or model_type is not None                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AssertionError: Different number of candidates and references

In [ ]:
# P, R, F1 = score(cands, refs, lang="en", verbose=True)

In [ ]:
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"]
  actual_spoiler_strip = dataframe.iloc[i]["actual_spoiler"]


  ref = word_tokenize(gen_spoiler_strip)
  hypo = word_tokenize(actual_spoiler_strip)

  print("\n")
  print(ref)
  print("\n")
  print(hypo)
  print("\n")



['tom', 'brady', "'", 's', 'definition', 'of', 'catching', 'up', 'involves', 'much', 'more', 'than', 'just', 'a', 'meal', '.', 'wes', 'welker', 'contacted', 'the', 'patriots', 'quarterback', 'friday', 'morning', 'on', 'a', 'miami', 'radio', 'station', '.', 'bra']


['how', 'about', 'that', 'morning', 'we', 'go', 'throw', '?']




['nasa', 'scientists', 'are', 'predicting', 'the', 'hole', 'in', 'the']


['2070']




['research', 'by', 'hiring', 'software', 'provider', 'cangrade', 'found', 'that', 'being', 'intellectually', 'stimulated']


['intellectual', 'stimulation']




['cal', 'newport', "'", 's', 'investigation', 'into', 'how', 'passionate', 'people', 'like', 'steve', 'jobs', 'really', 'got', 'started', 'as', 'well', 'as', 'what', 'scientists', 'say', 'predicts', 'happiness', 'and', 'fuels', 'great', 'accomplishment', 'newport', '.', 'ryan', 'holiday', 'author', 'of', 'ego', 'is', 'the', 'enemy', 'your', 'passion', 'may', 'be', 'the', 'very', 'enemy', '.', 'the', 'only', 'way', 

In [ ]:
bert_scores_P

In [ ]:
bert_scores_R

In [ ]:
bert_scores_F1